In [3]:
import os
import glob

import cv2
import json
import fiona
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import time
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

from torchvision import transforms
from torchvision import datasets, models, transforms, models

sns.set(font_scale=1.3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
train_image_paths = glob.glob('./data_transformed/train/image_*.png')

paths = ['./data_512x512/train/', './data_512x512/test/']
for path in paths:
    os.makedirs(path, exist_ok=True)
    
paths = ['./data_256x256/train/', './data_256x256/test/']
for path in paths:
    os.makedirs(path, exist_ok=True)
    
paths = ['./data_transformed/train/', './data_transformed/test/']
for path in paths:
    os.makedirs(path, exist_ok=True)

In [5]:
# size = (512, 512)
# for train_image_path in tqdm(train_image_paths):
#     mask_path = train_image_path.replace('image', 'mask')

#     image = plt.imread(train_image_path) * 255
#     mask = plt.imread(mask_path) * 255

#     image = cv2.resize(image, size).astype(np.uint8)
#     mask = cv2.resize(mask, size).astype(np.uint8)

#     image = Image.fromarray(image)
#     image.save(train_image_path.replace('data_transformed', 'data_512x512'))

#     mask = Image.fromarray(mask)
#     mask.save(mask_path.replace('data_transformed', 'data_512x512'))

In [6]:
# for train_image_path in tqdm(glob.glob('./data_256x256/train/image_*.png')):
#     mask_path = train_image_path.replace('image', 'mask')
    
#     image = plt.imread(train_image_path)
#     mask = plt.imread(mask_path)
#     print(image.shape)
#     break

In [7]:
from scipy.ndimage import distance_transform_edt as distance


def calc_dist_map(seg):
    res = np.zeros_like(seg)
    posmask = seg.astype(bool)

    if posmask.any():
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask

    return res

In [8]:
# distances = []
# for path in tqdm(glob.glob('./data_transformed/train/mask_*.png')):
#     mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
#     H, W = mask.shape
#     mask = np.concatenate((mask, np.zeros((16, W))), axis=0)
#     mask = np.concatenate((mask, np.zeros((H + 16, 8))), axis=1).astype(np.float32)
#     distances.append(calc_dist_map(mask))
    
# distances = np.array(distances).astype(np.float32)

In [12]:
# np.save('distances.npy', distances)

In [13]:
# images = []
# for path in tqdm(glob.glob('./data_transformed/train/image_*.png')):
#     image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     H, W, C = image.shape
#     image = np.concatenate((image, np.zeros((16, W, C))), axis=0)
#     image = np.concatenate((image, np.zeros((H + 16, 8, C))), axis=1).astype(np.uint8)
#     images.append(image)
    
# images = np.array(images).astype(np.uint8)

In [14]:
# np.save('images.npy', images)

In [8]:
class EyeDataset(Dataset):
    def __init__(self, data_folder):
        self.image_files = glob.glob(f'{data_folder}/*.png')
        self.to_tensor = transforms.ToTensor()
        
    def read_image(self, path):
        image = cv2.imread(str(path), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.array(image / 255, dtype=np.float32)
        return image
        
    def parse_polygon(self, coordinates, image_size): 
        mask = np.zeros(image_size, dtype=np.float32) 

        if len(coordinates) == 1: 
            points = [np.int32(coordinates)] 
            cv2.fillPoly(mask, points, 1) 
        else: 
            points = [np.int32([coordinates[0]])] 
            cv2.fillPoly(mask, points, 1) 
            for polygon in coordinates[1:]: 
                points = [np.int32([polygon])] 
                cv2.fillPoly(mask, points, 0) 

        return mask

    def parse_mask(self, shape, image_size):
        mask = np.zeros(image_size, dtype=np.float32)
        coordinates = shape['coordinates']
        if shape['type'] == 'MultiPolygon':
            for polygon in coordinates:
                mask = np.maximum(self.parse_polygon(polygon, image_size), mask) 
        else:
            mask = np.maximum(self.parse_polygon(coordinates, image_size), mask) 

        return mask
    
    def read_layout(self, path, image_size):
        with open(path, 'r', encoding='cp1251') as f:
            json_contents = json.load(f)

        mask = np.zeros(image_size, dtype=np.float32)

        if type(json_contents) == dict and json_contents['type'] == 'FeatureCollection':
            features = json_contents['features']
        elif type(json_contents) == list:
            features = json_contents
        else:
            features = [json_contents]

        for shape in features:
            mask = np.maximum(self.parse_mask(shape['geometry'], image_size), mask)
            
        return mask
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        image = self.read_image(image_path)
        
        json_path = image_path.replace('png', 'geojson')
        mask = self.read_layout(json_path, image.shape[:2])
        
        image, mask = self.to_tensor(image), torch.tensor(mask)
        return image, mask

In [10]:
# for i, (image, mask) in tqdm(enumerate(EyeDataset('./data/test')), total=len(train_image_paths)):
#     image, mask = (image.permute(1, 2, 0) * 255).numpy().astype(np.uint8), mask.numpy().astype(np.uint8)
    
#     image = Image.fromarray(image)
#     image.save(f'./data_transformed/test/image_{i + 1}.png')
    
#     mask = Image.fromarray(mask)
#     mask.save(f'./data_transformed/train/mask_{i + 1}.png')